In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 24.1 MB/s 
     |████████████████████████████████| 182 kB 59.4 MB/s 
     |████████████████████████████████| 7.6 MB 57.4 MB/s 


In [ ]:
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import RobertaTokenizer,BertTokenizer, AutoConfig, AutoModelForSequenceClassification, AutoTokenizer
from transformers import (
    AdamW,
    get_linear_schedule_with_warmup
)
output_dir = '/content/drive/MyDrive/fewshot_data/Closed_Bert/model_save_closed_Set/'

tokenizer = AutoTokenizer.from_pretrained(output_dir+'tokenizer/')
known_model = AutoModelForSequenceClassification.from_pretrained(output_dir)


# Copy the model to the GPU.
known_model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.15, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [ ]:
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import RobertaTokenizer,BertTokenizer, AutoConfig, AutoModelForSequenceClassification, AutoTokenizer
from transformers import (
    AdamW,
    get_linear_schedule_with_warmup
)


output_dir = '/content/drive/MyDrive/fewshot_data/M6/'

#tokenizer = AutoTokenizer.from_pretrained(output_dir+'tokenizer/')
unknown_model = AutoModelForSequenceClassification.from_pretrained(output_dir)


# Copy the model to the GPU.
unknown_model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.15, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [ ]:
import pandas as pd
import regex as re
import numpy as np
df = pd.read_csv("/content/drive/MyDrive/fewshot_data/ood_test_data_small.csv")
df = df[['real_label','review']]

In [ ]:
arr_real_labels = [i for i in range(100,10000)]
df['real_label'] = df['real_label'].replace(arr_real_labels,100)
df=df.loc[df.real_label.values<=100]

In [ ]:
# Create sentence and label lists
#
labels = df.real_label.values
sentence1 = df.review.values
#sentence1=sentences
#labels = dataset_filtered.real_label.values

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent1 in zip(sentence1):
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    #print(sent1)
    encoded_dict = tokenizer.encode_plus(
                        sent1[0],# Sentence to encode.
                        truncation=True,
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 512,           # Pad & truncate all sentences.
                        padding='max_length',
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                        return_overflowing_tokens=False,
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)



# Set the batch size.  
batch_size = 16

# Create the DataLoader.
prediction_data = TensorDataset(input_ids, attention_masks, labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [ ]:
# Prediction on test set

print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))

# Put model in evaluation mode
known_model.eval()

unknown_model.eval()

unknown_logits_total_array = []
known_logits_total_array = []

# Tracking variables 
unknown_predictions , unknown_true_labels = [], []
known_predictions , known_true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions.
      unknown_result = unknown_model(b_input_ids, 
                     token_type_ids=None, 
                     attention_mask=b_input_mask,
                     return_dict=True)
      known_result = known_model(b_input_ids, 
                     token_type_ids=None, 
                     attention_mask=b_input_mask,
                     return_dict=True)

  unknown_logits = unknown_result.logits
  known_logits = known_result.logits
  # Move logits and labels to CPU
  unknown_logits = unknown_logits.detach().cpu().numpy()
  unknown_logits[:,100] = unknown_logits[:,100] + 11
  unknown_logits_total_array.extend(unknown_logits.tolist())
  unknown_pred_labels = np.argmax(unknown_logits, axis=1)
  unknown_label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  unknown_predictions.extend(unknown_pred_labels.tolist())
  unknown_true_labels.extend(unknown_label_ids.tolist())

  # Move logits and labels to CPU
  known_logits = known_logits.detach().cpu().numpy()
  known_logits[:,100] = known_logits[:,100] + 0
  known_logits_total_array.extend(known_logits.tolist())
  known_pred_labels = np.argmax(known_logits, axis=1)
  known_label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  known_predictions.extend(known_pred_labels.tolist())
  known_true_labels.extend(known_label_ids.tolist())

print('    DONE.')

Predicting labels for 20,000 test sentences...
    DONE.


In [ ]:
from sklearn.metrics import classification_report

unknown_result_report= classification_report(unknown_true_labels, unknown_predictions, digits=5)
print(unknown_result_report)

known_result_report= classification_report(known_true_labels, known_predictions, digits=5)
print(known_result_report)

              precision    recall  f1-score   support

           0    0.75472   0.80000   0.77670       100
           1    0.50000   0.01000   0.01961       100
           2    0.94118   0.64000   0.76190       100
           3    0.94898   0.93000   0.93939       100
           4    0.63333   0.19000   0.29231       100
           5    0.82759   0.48000   0.60759       100
           6    0.73134   0.49000   0.58683       100
           7    1.00000   0.72000   0.83721       100
           8    0.62000   0.93000   0.74400       100
           9    0.72619   0.61000   0.66304       100
          10    0.91262   0.94000   0.92611       100
          11    0.61538   0.80000   0.69565       100
          12    0.58750   0.94000   0.72308       100
          13    0.87234   0.41000   0.55782       100
          14    0.66667   0.54000   0.59669       100
          15    0.78151   0.93000   0.84932       100
          16    0.97436   0.76000   0.85393       100
          17    0.90722   0

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classificat

In [ ]:
predictions=[]
for i in range(len(unknown_true_labels)):
  if unknown_predictions[i]==100:
    predictions.append(100)
  else :
    predictions.append(known_predictions[i])


result_report= classification_report(known_true_labels, predictions, digits=5)
print(result_report)

              precision    recall  f1-score   support

           0    0.68376   0.80000   0.73733       100
           1    1.00000   0.01000   0.01980       100
           2    0.92958   0.66000   0.77193       100
           3    0.93939   0.93000   0.93467       100
           4    0.39130   0.18000   0.24658       100
           5    0.60241   0.50000   0.54645       100
           6    0.73770   0.45000   0.55901       100
           7    0.91026   0.71000   0.79775       100
           8    0.73228   0.93000   0.81938       100
           9    0.72289   0.60000   0.65574       100
          10    0.92157   0.94000   0.93069       100
          11    0.58647   0.78000   0.66953       100
          12    0.63448   0.92000   0.75102       100
          13    0.78431   0.40000   0.52980       100
          14    0.67059   0.57000   0.61622       100
          15    0.86111   0.93000   0.89423       100
          16    0.98684   0.75000   0.85227       100
          17    0.87755   0

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
df2 = pd.read_csv("/content/drive/MyDrive/fewshot_data/ood_test_data_small.csv")
df2["N_plus_one_prediction"] = predictions
df2['N_plus_one_prediction'] = df2['N_plus_one_prediction'].replace([100], 'OOD')
combined_df = df2
number_of_known_labels = 100

In [ ]:
def compute_precision_recall_for_known_classes(given_label):
    TP = combined_df[(combined_df["real_label"] == given_label) & (combined_df["N_plus_one_prediction"] == given_label)].shape[0]
    FP = combined_df[(combined_df["real_label"] != given_label) & (combined_df["N_plus_one_prediction"] == given_label)].shape[0]
    FN = combined_df[(combined_df["real_label"] == given_label) & (combined_df["N_plus_one_prediction"] != given_label)].shape[0]
    if(TP == 0):
        P = 0
        R = 0
    else:
        P = round(100*TP/(TP + FP), 2)
        R = round(100*TP/(TP + FN),2)
    return P, R

def compute_precision_recall_for_OOD():
    TP = combined_df[(combined_df["real_label"] >= number_of_known_labels) & (combined_df["N_plus_one_prediction"] == "OOD")].shape[0]
    FP = combined_df[(combined_df["real_label"] < number_of_known_labels) & (combined_df["N_plus_one_prediction"] == "OOD")].shape[0]
    FN = combined_df[(combined_df["real_label"] >= number_of_known_labels) & (combined_df["N_plus_one_prediction"] != "OOD")].shape[0]
    if(TP == 0):
        P = 0
        R = 0
    else:
        P = 100*TP/(TP + FP)
        R = 100*TP/(TP + FN)
    return P, R

def compute_performance_metrics():
    from statistics import mean
    
    precisions = []
    recalls = []
    for known_label in range(number_of_known_labels):
        p, r = compute_precision_recall_for_known_classes(known_label)
        precisions.append(p)
        recalls.append(r)
    
    precision_known, recall_known = round(mean(precisions),2), round(mean(recalls),2)
    f1_known = round(2*precision_known*recall_known/(precision_known + recall_known),2)
    
    precision_ood, recall_ood = compute_precision_recall_for_OOD()
    f1_ood = round(2*precision_ood*recall_ood/(precision_ood + recall_ood),2)
    
    precisions.append(precision_ood)
    recalls.append(recall_ood)
    
    precision, recall = round(mean(precisions),2), round(mean(recalls),2)
    f1 = round(2*precision*recall/(precision + recall),2)
    
    return {
        "known": {
            "precision": precision_known,
            "recall": recall_known,
            "f1": f1_known,
        },
        "ood": {
            "precision": round(precision_ood,2),
            "recall": round(recall_ood,2),
            "f1": f1_ood,
        },
        "overall": {
            "precision": precision,
            "recall": recall,
            "f1": f1,
        }
        
    }

def print_performance_metrics():
    performance_dict = compute_performance_metrics()
    print("Precision  Recall f1")
    print("Known:",performance_dict["known"]["precision"],performance_dict["known"]["recall"],performance_dict["known"]["f1"], end = " \n")
    print("odd:",performance_dict["ood"]["precision"],performance_dict["ood"]["recall"],performance_dict["ood"]["f1"], end = " \n")
    print("overall:",performance_dict["overall"]["precision"],performance_dict["overall"]["recall"],performance_dict["overall"]["f1"], end = " \n")


In [ ]:
print_performance_metrics()

Precision  Recall f1
Known: 67.59 56.38 61.48 
odd: 68.0 83.76 75.06 
overall: 67.6 56.65 61.64 
